# Step 2: Viterbi algorithm

In [1]:
import matplotlib.pyplot as plt
from random import randint

We begin by reloading the Data_Munging file

In [2]:
%run 'Data_Munging.ipynb'

<class 'pandas.tslib.Timedelta'>
61.0


In [3]:
transitionDistribution

,0,1,4,5,10,13,15,17
0,0.965755,0.000974,0.212329,0.095041,0.001033,0.222222,0.014493,0.475
1,0.006473,0.998539,0.010274,0.000000,0.000000,0.000000,0.000000,0.000
4,0.014408,0.000398,0.609589,0.000000,0.002928,0.000000,0.005797,0.000
5,0.003550,0.000000,0.020548,0.904959,0.000000,0.000000,0.000000,0.000
10,0.003132,0.000000,0.113014,0.000000,0.995867,0.000000,0.000000,0.000
13,0.002297,0.000000,0.020548,0.000000,0.000172,0.777778,0.000000,0.025
15,0.001462,0.000089,0.010274,0.000000,0.000000,0.000000,0.965217,0.000
17,0.002923,0.000000,0.003425,0.000000,0.000000,0.000000,0.014493,0.500


In [4]:
observationDistribution

,0,1,4,5,10,13,15,17
1,0.023805,0.001284,0.000000,0.000000,0.000000,0.088889,0.092754,0.025
5,0.098350,0.002744,0.041096,0.859504,0.001378,0.000000,0.000000,0.000
6,0.123617,0.001859,0.386986,0.066116,0.000000,0.011111,0.000000,0.025
7,0.008979,0.000000,0.000000,0.000000,0.000000,0.011111,0.002899,0.075
8,0.170390,0.000177,0.020548,0.000000,0.000000,0.222222,0.142029,0.675
9,0.053456,0.000266,0.003425,0.000000,0.000000,0.055556,0.156522,0.000
12,0.065776,0.985172,0.037671,0.000000,0.000000,0.000000,0.002899,0.025
13,0.031948,0.002966,0.000000,0.004132,0.000000,0.000000,0.017391,0.050
14,0.190436,0.001903,0.345890,0.066116,0.000431,0.077778,0.005797,0.025
17,0.029651,0.000266,0.000000,0.000000,0.000000,0.122222,0.144928,0.050


In [27]:
initialDistribution = Series([0,0,0.2,0,0.8,0,0,0], index = [0,1,4,5,10,13,15,17])
initialDistribution

0     0.0
1     0.0
4     0.2
5     0.0
10    0.8
13    0.0
15    0.0
17    0.0
dtype: float64

In [28]:
sensorLabel

1              Microwave
5       Hall-Toilet door
6     Hall-Bathroom door
7          Cups cupboard
8                 Fridge
9        Plates cupboard
12            Front door
13            Dishwasher
14           ToiletFlush
17               Freezer
18         Pans Cupboard
20       Washing machine
23    Groceries Cupboard
24     Hall-Bedroom door
dtype: object

In [29]:
observation = Series([24,24,5,6,14,14,6,9,9,8,8,1,23,1,23,13,9,18,7,9,5,5,12,12,8,23,8,6,14,6,18,18,18,12,18,8,8,7,7,7,8,8,6,6,14,6])
observation

0     24
1     24
2      5
3      6
4     14
5     14
6      6
7      9
8      9
9      8
10     8
11     1
12    23
13     1
14    23
15    13
16     9
17    18
18     7
19     9
20     5
21     5
22    12
23    12
24     8
25    23
26     8
27     6
28    14
29     6
30    18
31    18
32    18
33    12
34    18
35     8
36     8
37     7
38     7
39     7
40     8
41     8
42     6
43     6
44    14
45     6
dtype: int64

In [30]:
observation[5]

14

In [31]:
initialDistribution

0     0.0
1     0.0
4     0.2
5     0.0
10    0.8
13    0.0
15    0.0
17    0.0
dtype: float64

In [32]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [33]:
def Viterbi(observation, activityLabel, initialDistribution, observationDistribution, transitionDistribution):
    probabilityTable = pd.DataFrame(0.0, index = activityLabel.index, columns = observation.index)
    path = {}
    for activityIndex in activityLabel.index.values:
        probabilityTable[0][activityIndex] = initialDistribution[activityIndex] * observationDistribution[activityIndex][observation[0]]
        path[activityIndex] = [activityIndex]
    
    for observationIndex in range(1,len(observation)):
        newPath = {}
        for activityIndex in activityLabel.index.values:
            (proba, previousActivityInd) = max((probabilityTable[observationIndex-1][previousActivityIndex]*transitionDistribution[previousActivityIndex][activityIndex]*observationDistribution[activityIndex][observation[observationIndex]], previousActivityIndex) for previousActivityIndex in activityLabel.index.values )
            #print proba 
            probabilityTable[observationIndex][activityIndex] = proba
            newPath[activityIndex] = path[previousActivityInd] + [activityIndex]
        path = newPath
    
    (finalProba, finalActivityIndex) = max((probabilityTable[len(observation)-1][activityIndex], activityIndex) for activityIndex in activityLabel.index.values )
    return finalProba, path[finalActivityIndex]
        
        
        

Now we will create the testing set:

In [39]:
day25Sensor = [sensorData['Start time'][x].day == 27 for x in range(1,len(sensorData))]
day25Sensor = day25Sensor + [False]
day25SensorData = sensorData[day25Sensor]
day25SensorData.index = np.arange(1,len(day25SensorData)+1)

In [42]:
day25Activity = [activityData['Start time'][x].day == 27 for x in range(1,len(activityData))]
day25Activity = day25Activity + [False]
day25ActivityData = activityData[day25Activity]
day25ActivityData.index = np.arange(1,len(day25ActivityData)+1)


In [51]:
testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(day25SensorData,day25ActivityData,60)

IndexError: index 1127 is out of bounds for axis 0 with size 1094

In [16]:
testingObservation = Series(0, index = testingFeatureMatrix.columns)
for i in range(len(testingObservation)): 
    r = np.where(testingFeatureMatrix[i] == 1)
    result = testingFeatureMatrix.index[r[0]]
    testingObservation[i] = result[0]

numOfObservation = len(testingObservation)
obs = testingObservation[:numOfObservation]
validatedPath = testingLabel[:numOfObservation]

In [17]:
proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)


In [18]:
#error rate
errorRate = ( sum(validatedPath != path)/float(len(validatedPath)) ) * 100 
errorRate

26.652298850574713

In [35]:
errorRate = []
numOfEvents = []
for x in range(1,len(testingObservation),100): 
    obs = testingObservation[:x]
    validatedPath = testingLabel[:x]
    proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)
    errorRate.append(  (sum(validatedPath != path)/float(len(validatedPath))) * 100 )
    numOfEvents.append(x)
    plt.plot(numOfEvents,errorRate)
    plt.xlabel("Number of observations in the training sequence")
    plt.ylabel("Error rate (%)")
    

In [36]:
plt.show()

In [21]:
def crossValidation():
    dayList = [26, 25]
    for d in dayList:
        print d
        daySensor = [sensorData['Start time'][x].day == d for x in range(1,len(sensorData))]
        daySensor = daySensor + [False]
        daySensorData = sensorData[daySensor]
        daySensorData.index = np.arange(1,len(daySensorData)+1)
        print daySensorData.index
        
        dayActivity = [activityData['Start time'][x].day == d for x in range(1,len(activityData))]
        dayActivity = dayActivity + [False]
        dayActivityData = activityData[dayActivity]
        dayActivityData.index = np.arange(1,len(dayActivityData)+1)
        print dayActivityData.index
        
        testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)
        
        testingObservation = Series(0, index = testingFeatureMatrix.columns)
        for i in range(len(testingObservation)): 
            r = np.where(testingFeatureMatrix[i] == 1)
            result = testingFeatureMatrix.index[r[0]]
            testingObservation[i] = result[0]
            
        errorRate = []
        numOfEvents = []
        for x in range(1,len(testingObservation),250): 
            obs = testingObservation[:x]
            validatedPath = testingLabel[:x]
            proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)
            errorRate.append(  (sum(validatedPath != path)/float(len(validatedPath))) * 100 )
            numOfEvents.append(x)
            plt.plot(numOfEvents,errorRate)
         
    plt.xlabel("Number of observations in the training sequence")
    plt.ylabel("Error rate (%)")
    plt.show()

In [22]:
crossValidation()

26
Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52, 53, 54, 55, 56, 57, 58, 59],
           dtype='int64')
Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')
25
Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
            69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
            86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98],
           dtype='int64')
Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')


In [50]:
sensorData.head()

,Start time,End time,ID,Val
1,2008-02-25 00:20:14,2008-02-25 00:22:57,24,1
2,2008-02-25 09:33:41,2008-02-25 09:33:42,24,1
3,2008-02-25 09:36:43,2008-02-25 09:37:04,5,1
4,2008-02-25 09:37:20,2008-02-25 09:37:23,6,1
5,2008-02-25 09:37:51,2008-02-25 09:37:52,14,1
